In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt #trabalhar com datas 
import matplotlib.pyplot as plt # graficos
import seaborn as sn

#Modelo
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier



In [ ]:
base = pd.read_csv('../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv') 
base.head()

# Descrição das variáveis 

* **sl_n = Serial Number**
* **gender = Gender- Male='M',Female='F**'
* **ssc_p = Secondary Education percentage- 10th Grade**
* **sc_b = Board of Education- Central/ Others**
* **hsc_p = Higher Secondary Education percentage- 12th Grade**
* **hsc_b = Board of Education- Central/ Others**
* **hsc_s = Specialization in Higher Secondary Education**
* **degree_p = Degree Percentage**
* **degree_t = Under Graduation(Degree type)- Field of degree education**
* **workex = Work Experience**
* **etest_p = Employability test percentage ( conducted by college)**
* **specialisation = Post Graduation(MBA)- Specialization**
* **mba_p = MBA percentage**
* **status = Status of placement- Placed/Not placed**
* **salary = Salary offered by corporate to candidates**

In [ ]:
base.info()
base.describe()

In [ ]:
#Valores nulos: valores nulos no salary indicam que a pessoa em questão não foi contratada, logo essas linhas não podem ser removidas 
#base.isnull().sum()   
#base.dropna(inplace = True)

In [ ]:
#Dados categóricos 
base.nunique()
base.head()
base['status'].value_counts()    # muitas observações a mais na class 1 em comparação a classe 0 
#base['status'].value_counts()['Placed']


# Relação entre experiência de trabalho e ser contratado 

In [ ]:
#utilizando o dicionário e a função "replace" para transformar em numericos
mapeamento_classes = {"workex": {"No": 0, "Yes": 1},
                "status": {"Placed": 1,"Not Placed": 0 }} 

base.replace(mapeamento_classes, inplace=True)


In [ ]:
#base['status'] = base['status'].astype('category')
#base['workex'] = base['workex'].astype('category')


plotar = base.groupby(['status','workex'])['status'].count().unstack('workex')#.fillna(0)
plotar[[0, 1]].plot(kind='bar', stacked=True) 
plt.title('Relação entre experiência (workex) de trabalho e ser contratado (status) ') 
plt.xlabel('Status') 
plt.ylabel('Frequência')


Preparando dados para classificação 

In [ ]:
#Preparando dados 
x = base['workex'].values
x_reshaped = x.reshape(-1,1)
y = base['status'].values

# Separando dados em treino e  teste 
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y, random_state = 1)

In [ ]:
x_train

Random Forest 

In [ ]:
#Random Forest 
rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)

In [ ]:
#previsão 
rfc_pred = rfc.predict(x_test) 

#metricas 
print(classification_report(y_test, rfc_pred))

As métricas para a classe 0 estão realmente ruins. Vamos analisar o que houve na matrix de confusão 

In [ ]:
#Matrix de confusão 
cm = confusion_matrix(y_test, rfc_pred) #gera a matriz de confusão
df_cm = pd.DataFrame(cm, index = [i for i in "01"],columns = [i for i in "01"]) #cria o df com as classes
plt.figure(figsize = (10,7)) #indica o tamanho da figura 
sn.heatmap(df_cm, annot=True) #plota a figura

Pode-ser perceber que o algoritmo classificou todos os objetos como sendo da classe 1, ou seja, como se todas as pessoas fossem contratadas. Isso pode ter ocorrido em funsão do desbalanceamento do dataset.A 
classe 1 (placed) tem  148 objetos, enqaunto a classe 0 (not placed)  tem apenas 67 objetos 